<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/main/Webscraper3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from bs4 import BeautifulSoup as b4
import asyncio
from google.colab import files
import multiprocessing
import os
import requests
import time
import zipfile
!pip install selenium

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [31]:
TRIAL = 3

### Parse chapter

In [32]:
def parseFile(html):
  #with open(html, "r") as file:
  soup = b4(html, features="html.parser")
  #print(soup.chapter-title.get_text())
  tag = soup.article.find_all('span')
  for t in tag:
    if "Chapter" in t.get_text():
      title = t.get_text() + ".html"
  soup = soup.find_all('p')
  with open(title, "w" , encoding='utf-8') as file:
    for paragraph in soup:
        file.write(str(paragraph) + "\n")  # Write each paragraph to the file
    print(str(soup))
  file.close()

### Download main webpage

In [33]:
def getChapter(url, browser, trial):
  try:
      browser.get(url)
      print(f"attempting to download url: {url}")
      timeout_in_seconds = 10
      WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
      html = browser.page_source
      parseFile(html)
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        file.close()
        print("I give up...")
      else:
        print(f"Download failed trial {trial}")
        getChapter(url, browser, trial + 1)

### Get webpage

In [34]:
def getNovelPage(url, browser, trial):
  try:
      print(f"getNovelPage trying url: {url}...")
      browser.get(url)
      html = browser.page_source
      soup = b4(html, features="html.parser")
      strainer = soup.find(class_="chapter-list").find_all('a')
      for link in strainer:
        getChapter("https://www.lightnovelpub.com"+ link['href'], browser, 0)
      return html
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        return -1
        file.close()
        print("I give up...")
      else:
        print(f"Download failed trial {trial}")
        getNovelPage(url, browser, trial + 1)
  finally:
      browser.quit()

### Get links and go to next chapter page, if avaliable (testing from file)

In [35]:
def getChapterPages(url, page_num):
  # Start a new selenium process
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  browser = webdriver.Firefox(options=options)

  # Get the novel main page
  url_source = getNovelPage(url, browser, 0)

  if url_source == -1:
    print("Unable to find page source!")
    return

  soup = b4(url_source, features="html.parser")
  links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")
  next_url = ""
  for link in links:
    next_url =  link.find('a')['href']
  if next_url:
    # Make new url
    new_url = "https://www.lightnovelpub.com" + next_url
    getChapterPages(new_url, page_num+1)
  else:
    browser.quit()

### Input url

In [36]:
url = input("Enter URL: ")
title = input("Enter directory: ")
directory = title
os.mkdir(directory)
os.chdir(directory)
getChapterPages(url, 1)

Enter URL: https://www.lightnovelpub.com/novel/i-became-a-genius-mage-in-the-cthulhu-game/chapters
Enter directory: I became a genius mage
getNovelPage trying url: https://www.lightnovelpub.com/novel/i-became-a-genius-mage-in-the-cthulhu-game/chapters...
attempting to download url: https://www.lightnovelpub.com/novel/i-became-a-genius-mage-in-the-cthulhu-game-1679/chapter-1
[<p>[Hidden Ending Perk!]</p>, <p>[The hidden class, &lt;Mage Gazing into the Abyss&gt; has been unlocked.]</p>, <p>A dry message appeared on the monitor.</p>, <p>“Ugh…”</p>, <p>I did it!</p>, <p>“A hidden class! You did it, Kim Shin-hwa!”</p>, <p>[Cthulhu World].</p>, <p>This is the game I’ve dedicated over 1,000 hours to recently.</p>, <p>[Cthulhu World] is a game set in modern Korea, incorporating elements of the Cthulhu mythos like the “Mountains of Madness” “Dagons” and “Nyarlathotep”.</p>, <p>Therefore, everything in this game existed in a way that was more grim and horrifying than reality.</p>, <p>The civiliz

In [42]:
directory = os.getcwd()

# List all files in the directory
all_files = os.listdir(directory)

# Specify the name of the zip file to create
zip_file_name = title + ".zip"

# Create a zip file
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    # Add each file in the directory to the zip file
    for file in all_files:
        file_path = os.path.join(directory, file)
        zipf.write(file_path, os.path.basename(file_path))

print("Zip file created:", zip_file_name)

files.download(directory + '/' + zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

RuntimeError: File size unexpectedly exceeded ZIP64 limit